In [ ]:
import functools
import time

from IPython.display import HTML, Image 
import gym

try:
  import brax
except ImportError:
  from IPython.display import clear_output 
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax import envs
from brax import jumpy as jp
from brax.envs import to_torch
from brax.io import html
from brax.io import image
import jax
from jax import numpy as jnp
import torch
v = torch.ones(1, device='cuda')  # init torch cuda before jax


In [ ]:

environment = "mydrone"  # @param ['ant', 'halfcheetah', 'hopper', 'humanoid', 'reacher', 'walker2d', 'fetch', 'grasp', 'ur5e']
env = envs.create(env_name=environment)
state = env.reset(rng=jp.random_prngkey(seed=0))

In [ ]:

HTML(html.render(env.sys, [state.qp]))


In [ ]:

%%time
rollout = []
for i in range(100):
  # wiggle sinusoidally with a phase shift per actuator
  # To change: action
  action = jp.sin(i * jp.pi / 15 + jp.arange(0, env.action_size) * jp.pi)
  state = env.step(state, action)
  rollout.append(state)
# jit compile env.step:
jit_env_step = jax.jit(env.step)
state = jit_env_step(state, jnp.ones((env.action_size,)))


In [ ]:

%%time
for _ in range(100):
  state = jit_env_step(state, jnp.ones((env.action_size,)))


In [ ]:
HTML(html.render(env.sys, [s.qp for s in rollout]))